In [ ]:
!pip3 install fse

     |████████████████████████████████| 112kB 13.3MB/s 
     |████████████████████████████████| 24.2MB 1.6MB/s 
     |████████████████████████████████| 32.8MB 76kB/s 
     |████████████████████████████████| 5.0MB 48.3MB/s 
     |████████████████████████████████| 276kB 40.2MB/s 
  Created wheel for fse: filename=fse-0.1.15-cp36-cp36m-linux_x86_64.whl size=255579 sha256=17d2a17b39e22d161c3bf9f93cdcecedf4e97ad04ec0ef04de635a763ef5b46e
  Stored in directory: /root/.cache/pip/wheels/c9/f9/00/f323d92e0091436ce32f54cb7760b8a229469a184f940c2a4d
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817239 sha256=7ae5a98da8f59fb27048cc6b1cddef67c6d8186951bc723443d2d0f8a5b550d0
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.1.0-cp36-none-any.whl size=5068034 sha256=174a402cbf89ba37a5058c68a89b10a166f131e63e64a24b2f1ee425919ccb82
  Stored in directory: /root/.cac

In [ ]:
import os
!git clone -l -s https://github.com/harpreetvirkk/NLP-QuestionAnswerSystem rep
%cd rep
!ls dataset

Cloning into 'rep'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 165 (delta 20), reused 153 (delta 17), pack-reused 0
Receiving objects: 100% (165/165), 1.69 MiB | 22.53 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/rep
S08_question_answer_pairs.txt  S10_question_answer_pairs.txt  text_data_toc.csv
S09_question_answer_pairs.txt  text_data


In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
import fse
from fse import SplitIndexedList
import gensim.downloader as api
from fse.models import uSIF
from sklearn import metrics
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import logging
# logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
df1 = pd.read_csv('./dataset/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('./dataset/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('./dataset/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')
frames = [df1, df2, df3]
df = pd.concat(frames)

In [ ]:
def getArticleText(file):
  fpath = './dataset/text_data/'+file+'.txt.clean'
  try:
    f = open(fpath, 'r')
    text = f.read()
  except UnicodeDecodeError:
    f = open(fpath, 'r', encoding = 'ISO-8859-1')
    text = f.read()
  return text

In [ ]:
df = df.dropna(subset=['ArticleFile'])
df['ArticleText'] = df['ArticleFile'].apply(lambda x: getArticleText(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: re.sub(r'(\n)+', '. ', x))
df = df.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis='columns')
df

ArticleTitle  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
# stop_words = set(stopwords.words('english'))
def cleanQuestion(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanAnswer(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanText(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s\.\?]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

In [ ]:
df['Question'] = df['Question'].apply(lambda x: cleanQuestion(x))
df['Answer'] = df['Answer'].apply(lambda x: cleanAnswer(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: cleanText(x))
df

ArticleTitle  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
allQuestion = '. '.join(list(df['Question']))
allAnswer = '. '.join(list(df['Answer']))
allContext = '. '.join(list(df['ArticleText']))
text = allQuestion + allAnswer + allContext
s = SplitIndexedList(text.split('.'))

In [ ]:
glove = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
from fse.models import uSIF
model = uSIF(glove, workers=2, lang_freq="en")

In [ ]:
model.train(s)

(1209160, 18383309)

In [ ]:
model.sv.similar_by_word("abraham", wv=glove, indexable=s.items)

[(' abraham lincoln', 21570, 0.9007503986358643),
 (' abraham lincoln', 12064, 0.9007503986358643),
 (' abraham lincoln', 14101, 0.9007503986358643),
 (' abraham lincoln', 24286, 0.9007503986358643),
 (' abraham lincoln', 22928, 0.9007503986358643),
 (' abraham lincoln', 10706, 0.9007503986358643),
 (' abraham lincoln', 28360, 0.9007503986358643),
 (' abraham lincoln', 15459, 0.9007503986358643),
 (' abraham lincoln', 13422, 0.9007503986358643),
 (' abraham lincoln', 19533, 0.9007503986358643)]

In [ ]:
q = 'was abraham lincoln the sixteenth president of the united states'
c1 = 'Abraham Lincoln (February 12, 1809 – April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination.'
c2 = 'Scholars now rank Lincoln among the top three U.S. Presidents, with the majority of those surveyed placing him first.'
c3 = 'Symbolic log cabin at Abraham Lincoln Birthplace National Historic Site.'

tmpq = (q.split(), 0)
tmpc1 = (c1.split(), 0)
tmpc2 = (c2.split(), 0)
tmpc3 = (c3.split(), 0)

print(metrics.pairwise.cosine_similarity(model.infer([tmpq]), model.infer([tmpc1])))
print(metrics.pairwise.cosine_similarity(model.infer([tmpq]), model.infer([tmpc2])))
print(metrics.pairwise.cosine_similarity(model.infer([tmpq]), model.infer([tmpc3])))

[[0.5035991]]
[[0.34502378]]
[[0.1622887]]


In [ ]:
df['Question'].iloc[0]

'was abraham lincoln the sixteenth president of the united states'

In [ ]:
def getSim(q, x):
  x = (str(x).split(), 0)
  sim = metrics.pairwise.cosine_similarity(model.infer([q]), model.infer([x]))
  return sim

def getAnswer(question, context):
  q = (str(question).split(), 0)
  c = pd.DataFrame(str(context).split('.'))
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  max = c.sort_values(by='sim', ascending=False).iloc[:3]
  return max

In [ ]:
qno = 250
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])

Question:  in 1905 coolidge met and married whom
Top Solutions: 



,0,sim
61,in 1905 coolidge met and married grace anna goodhue a local schoolteacher and fellow vermonter,[[0.84691125]]
69,they had two sons john coolidge born in 1906 and calvin jr,[[0.82305527]]
57,in 1897 coolidge was admitted to the bar,[[0.75742567]]


In [ ]:
qno = 100
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
sol = getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])

Question:  was amedeo avogadro born in turin august 9th 1776 to a noble ancient family of piedmont italy
Top Solutions: 



In [ ]:
qno = 1234
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])

Question:  how much time to penguins spend on land
Top Solutions: 



,0,sim
32,this is also the distance that antarctic tourists are told to keep from penguins tourists are not supposed to approach closer than 3 meters but are not expected to withdraw if the penguins come closer,[[0.78479326]]
26,they spend half of their life on land and half in the oceans,[[0.7688274]]
83,isabelline penguins tend to live shorter lives than normal penguins as they are not well camouflaged against the deep and are often passed over as mates,[[0.75811774]]


In [ ]:
qno = 1
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])

Question:  was abraham lincoln the sixteenth president of the united states
Top Solutions: 



,0,sim
619,lincoln illinois is the only city to be named for abraham lincoln before he became president,[[0.84491354]]
0,abraham lincoln,[[0.8129417]]
287,photograph showing the march 4 1861 inauguration of abraham lincoln in front of united states capitol,[[0.7974195]]


In [ ]:
qno = 600
print('Question: ', df['Question'].iloc[qno])
print('Top Solutions: \n')
getAnswer(df['Question'].iloc[qno], df['ArticleText'].iloc[qno])

Question:  was ford the last surviving member of the warren commission
Top Solutions: 



,0,sim
642,ford was the last surviving member of the warren commission,[[1.0]]
201,johnson appointed ford to the warren commission a special task force set up to investigate the assassination of president john f,[[0.8035106]]
253,gerald ford is sworn in as the 38th president of the united states by chief justice warren burger in the white house east room while betty ford looks on,[[0.79951906]]
